In [3]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
import os

# Caminho para a pasta de CSVs
csv_folder_path = '../Database/Questao3/Repositorios'

# Carregar a lista de repositórios do CSV
repos_list_csv = pd.read_csv('../Database/Resultados-Busca/repositories.csv')

# Supondo que a coluna com o nome dos repositórios seja 'repo_name'
# Substituir '/' por '_' e adicionar o sufixo para coincidir com o padrão dos arquivos
repos_list = {repo.replace("/", "_") + "_package_json_history" for repo in repos_list_csv['name'].tolist()}

# Obter a lista de arquivos na pasta de CSVs (removendo a extensão .csv)
generated_csvs = {os.path.splitext(file)[0] for file in os.listdir(csv_folder_path) if file.endswith('.csv')}
repositorios_list = list(generated_csvs)
# Substituindo "_package_json_history" e "_" por "/" em cada elemento da lista
repositorios_list_modified = [repo.replace("_package_json_history", "").replace("_", "/") for repo in repositorios_list]

# Lista para armazenar dados de análise final
final_analysis_data = []

# Função para salvar o gráfico
def save_plot(x, y1, y2, x_label, title, filename, plot_type):
    plt.figure(figsize=(14, 7))
    plt.plot(x, y1, label='Dependências', marker='o', color='blue', markersize=8)
    plt.plot(x, y2, label='DevDependências', marker='o', color='orange', markersize=8)

    plt.title(title, fontsize=16)
    plt.xlabel(x_label, fontsize=14)
    plt.ylabel('Número de Dependências', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), frameon=True, framealpha=1, edgecolor='black', facecolor='white')
    plt.tight_layout()

    # Definir o diretório de salvamento com base no tipo de gráfico
    if plot_type == 'date':
        save_path = "../Image/Questao3/Date"
    elif plot_type == 'commit':
        save_path = "../Image/Questao3/Commit"
    else:
        raise ValueError("Tipo de gráfico inválido. Use 'date' ou 'commit'.")

    os.makedirs(save_path, exist_ok=True)  # Cria o diretório se não existir
    plt.savefig(f"{save_path}/{filename}.png")
    plt.close()  # Fecha a figura para liberar memória

# Iterar sobre cada repositório listado
for i in range(len(repositorios_list)):  # Garante que não ultrapasse o tamanho da lista
    repo_name = repositorios_list[i]
    name = repositorios_list_modified[i]
    # Carregar os dados
    path = f'{csv_folder_path}/{repo_name}.csv'

    if not os.path.exists(path):
        print(f"Arquivo não encontrado: {path}")
        continue  # Pula para o próximo repositório se o arquivo não existir

    df = pd.read_csv(path)

    # Verificar se as colunas necessárias existem
    if not all(col in df.columns for col in ['date', 'commit', 'dependencies_changes', 'dev_dependencies_changes']):
        print(f"Colunas necessárias ausentes no arquivo: {repo_name}.csv")
        continue

    # Converter a coluna 'date' para o tipo datetime
    df['date'] = pd.to_datetime(df['date'])

    # Criar conjuntos para rastrear todas as bibliotecas
    all_dependencies = set()
    all_dev_dependencies = set()

    # Criar um DataFrame para armazenar a evolução
    evolution_data = []

    # Função para converter string em dicionário com tratamento de exceção
    def safe_literal_eval(s):
        try:
            return ast.literal_eval(s)
        except (ValueError, SyntaxError):
            return {'added': [], 'removed': []}

    df['dependencies_changes'] = df['dependencies_changes'].apply(safe_literal_eval)
    df['dev_dependencies_changes'] = df['dev_dependencies_changes'].apply(safe_literal_eval)

    # Iterar sobre as mudanças
    for index, data_row in df.iterrows():
        # Adicionando mudanças de dependências
        dependencies_added = data_row['dependencies_changes'].get('added', [])
        dependencies_removed = data_row['dependencies_changes'].get('removed', [])

        dev_dependencies_added = data_row['dev_dependencies_changes'].get('added', [])
        dev_dependencies_removed = data_row['dev_dependencies_changes'].get('removed', [])

        # Atualizar conjuntos de bibliotecas
        all_dependencies.update(dependencies_added)
        all_dependencies.difference_update(dependencies_removed)

        all_dev_dependencies.update(dev_dependencies_added)
        all_dev_dependencies.difference_update(dev_dependencies_removed)

        # Armazenar evolução
        evolution_data.append({
            'commit': data_row['commit'],
            'date': data_row['date'],
            'current_dependencies': list(all_dependencies),
            'current_dev_dependencies': list(all_dev_dependencies),
            'current_dependencies_count': len(all_dependencies),
            'current_dev_dependencies_count': len(all_dev_dependencies),
        })

    # Criar DataFrame de evolução
    evolution_df = pd.DataFrame(evolution_data)

    # Gráfico da evolução das dependências ao longo do tempo
    save_plot(evolution_df['date'], evolution_df['current_dependencies_count'], evolution_df['current_dev_dependencies_count'],
              'Data', f'Evolução das Dependências e DevDependências ao longo do Tempo - ({name})', f'{repo_name}_evolucao_dependencias_tempo', 'date')

    # Gráfico da evolução das dependências ao longo dos commits
    save_plot(evolution_df['commit'], evolution_df['current_dependencies_count'], evolution_df['current_dev_dependencies_count'],
              'Commits', f'Evolução das Dependências e DevDependências ao longo dos Commits - ({name})', f'{repo_name}_evolucao_dependencias_commits', 'commit')

    # Análise final
    total_unique_dependencies = len(all_dependencies)
    total_unique_dev_dependencies = len(all_dev_dependencies)

    print(f'Total de dependências únicas: {total_unique_dependencies}')
    print(f'Total de devDependências únicas: {total_unique_dev_dependencies}')

    # Verificar se as mesmas bibliotecas estiveram presentes sempre
    if not evolution_df['current_dependencies'].empty:
        always_present_dependencies = set.intersection(*(set(x) for x in evolution_df['current_dependencies']))
    else:
        always_present_dependencies = set()

    if not evolution_df['current_dev_dependencies'].empty:
        always_present_dev_dependencies = set.intersection(*(set(x) for x in evolution_df['current_dev_dependencies']))
    else:
        always_present_dev_dependencies = set()

    print(f'Dependências sempre presentes: {always_present_dependencies}')
    print(f'DevDependências sempre presentes: {always_present_dev_dependencies}')

    # Armazenar dados de análise final
    final_analysis_data.append({
        'repo_name': repo_name,
        'total_unique_dependencies': total_unique_dependencies,
        'total_unique_dev_dependencies': total_unique_dev_dependencies,
        'always_present_dependencies': ', '.join(always_present_dependencies),
        'always_present_dev_dependencies': ', '.join(always_present_dev_dependencies)
    })


# Criar um DataFrame com os dados de análise final
final_analysis_df = pd.DataFrame(final_analysis_data)

# Salvar o DataFrame como um arquivo CSV
final_analysis_df.to_csv('../Database/Questao3/final_analysis.csv', index=False)

Total de dependências únicas: 7
Total de devDependências únicas: 5
Dependências sempre presentes: {'react', 'react-native'}
DevDependências sempre presentes: set()
Total de dependências únicas: 0
Total de devDependências únicas: 44
Dependências sempre presentes: set()
DevDependências sempre presentes: set()
Total de dependências únicas: 20
Total de devDependências únicas: 29
Dependências sempre presentes: set()
DevDependências sempre presentes: {'style-loader', 'rucksack-css', 'sass-loader', 'autoprefixer'}
Total de dependências únicas: 12
Total de devDependências únicas: 17
Dependências sempre presentes: {'assets', 'recorder.js', 'main.js', 'recorderjs', 'stats.js', 'main', 'recorder', 'dat.gui', 'howler'}
DevDependências sempre presentes: {'babel-preset-env', 'webpack-cli', 'eslint', 'babel-preset-react', 'clang-format', 'cross-env', 'eslint-config-google', 'babel-preset-es2015', '@tensorflow-models/posenet', 'babel-polyfill', 'babel-preset-es2017', 'parcel-bundler', 'babel-core', 'd

C:\Users\tochi\AppData\Local\Temp\ipykernel_6260\376146471.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


Total de dependências únicas: 5
Total de devDependências únicas: 36
Dependências sempre presentes: set()
DevDependências sempre presentes: {'react', 'react-test-renderer', 'react-dom', 'enzyme-to-json', 'react-scripts', 'enzyme'}
Total de dependências únicas: 0
Total de devDependências únicas: 31
Dependências sempre presentes: set()
DevDependências sempre presentes: {'@testing-library/dom', 'inquirer', 'prettier-plugin-tailwindcss', '@commitlint/cli', 'solid-js', '@commitlint/cz-commitlint', 'vite', 'typescript', 'turbo', 'babel-preset-solid', 'prettier', '@commitlint/config-conventional', '@babel/core', 'vite-plugin-solid', 'commitizen', '@babel/preset-env', '@testing-library/user-event', '@types/node', '@changesets/cli', '@testing-library/jest-dom'}
Total de dependências únicas: 1
Total de devDependências únicas: 49
Dependências sempre presentes: set()
DevDependências sempre presentes: {'typescript'}
Total de dependências únicas: 25
Total de devDependências únicas: 28
Dependências se

In [1]:
# Perform initial analysis to answer each research question.
import pandas as pd
# 1. Identificar as dependências mais comuns entre os projetos UI.
# Contar a ocorrência de cada dependência presente em "always_present_dependencies"
from collections import Counter

# Load the CSV file to examine its structure and content
file_path = '../Database/Questao3/final_analysis.csv'
data = pd.read_csv(file_path)

# Display the first few rows and column names to understand the structure of the dataset
data.head(), data.columns

# Flattening the dependencies list and counting occurrences
all_dependencies = data['always_present_dependencies'].dropna().str.split(',').sum()
dependency_counts = Counter([dep.strip() for dep in all_dependencies])

# Obter as dependências mais comuns
most_common_dependencies = dependency_counts.most_common(20)

# Flattening the dependencies list and counting occurrences
all_devdependencies = data['always_present_dev_dependencies'].dropna().str.split(',').sum()
devdependency_counts = Counter([dep.strip() for dep in all_devdependencies])

# Obter as dependências mais comuns
most_common_devdependencies = devdependency_counts.most_common(20)

# 2. Analisar a relação entre número de dependências e características (total dependencies as proxy for project size)
# Estatísticas descritivas para o total de dependências e de dependências de desenvolvimento
total_dependencies_stats = data[['total_unique_dependencies', 'total_unique_dev_dependencies']].describe()

# 3. Verificar a consistência de dependências (se houve mudanças significativas) ao longo dos commits
# Contagem de valores não nulos em 'always_present_dependencies' e 'always_present_dev_dependencies'
consistency_counts = data[['always_present_dependencies', 'always_present_dev_dependencies']].notnull().sum()

most_common_dependencies, most_common_devdependencies ,total_dependencies_stats, consistency_counts


([('react', 120),
  ('react-dom', 88),
  ('react-native', 37),
  ('react-scripts', 29),
  ('vue', 29),
  ('react-router-dom', 22),
  ('next', 22),
  ('axios', 21),
  ('lodash', 20),
  ('react-redux', 17),
  ('redux', 16),
  ('@testing-library/react', 16),
  ('@testing-library/jest-dom', 16),
  ('prop-types', 15),
  ('vue-router', 15),
  ('@testing-library/user-event', 15),
  ('express', 13),
  ('bootstrap', 13),
  ('web-vitals', 12),
  ('react-navigation', 11)],
 [('eslint', 156),
  ('typescript', 135),
  ('prettier', 94),
  ('jest', 67),
  ('webpack', 63),
  ('@types/react', 58),
  ('husky', 58),
  ('babel-loader', 52),
  ('@types/node', 52),
  ('eslint-plugin-import', 50),
  ('eslint-plugin-react', 50),
  ('autoprefixer', 46),
  ('react', 44),
  ('react-test-renderer', 44),
  ('css-loader', 43),
  ('@babel/core', 43),
  ('babel-eslint', 42),
  ('eslint-config-prettier', 41),
  ('babel-core', 40),
  ('rimraf', 39)],
        total_unique_dependencies  total_unique_dev_dependencies
 cou